In [3]:
import pandas as pd
import pickle
import json
import re
import ast

In [4]:
def extract_dict(string: str):
    """
    Extracts dictionary from the input string.

    Args:
        string (str): The input string.

    Returns:
        Union[str, None]: Extracted dictionary as a string, or None if no dictionary found.
    """
    start: int = string.find('{')
    end: int = string.find('}')
    if start != -1 and end != -1:
        return string[start:end+1]
    else: 
        return None

def preprocess_prompt(prompt,sep_context = True, lang = 'de'):
    
    prompt_split = prompt.split('Dies ist der zu annotierende Textausschnitt:')

    if len(prompt_split) < 2:
        prompt_split = prompt.split('Dies ist der Textausschnitt:')

    instruction_split = prompt_split[0].split('Gehe Schritt für Schritt vor:')

    if len(instruction_split) > 1:
        instruction = '\n' + 'Gehe Schritt für Schritt vor: ' + instruction_split[1] + '\n'
        prompt_split[0] = instruction_split[0]
    else: 
        instruction = ''

    prompt_split[0] = prompt_split[0].split('Hier einige Beispiele, wie die Aufgabe zu bearbeiten ist:')[0].strip()
    prompt_split[0] = prompt_split[0].split('Hier einige Beispiele, wie die Dictionaries aussehen sollen:')[0].strip()
    prompt_split[0] += instruction

    if lang == 'en':
        prompt_split[0] = translate_en(prompt_split[0])
        prompt_split[1] = translate_en(prompt_split[1])

    return prompt_split

def train_df_to_text(row, lang = 'de',simple_dict = False):

    prompt_split = preprocess_prompt(row['prompt'],lang='de')

    result = row['result']

    if simple_dict:
        prompt_split[0], result = simple_dict_prompt(prompt_split[0],result)

    if lang == 'en':
        training_lang = 'english'
    elif lang == 'de':
        training_lang = 'german'
    training_text = f'''Below is an instruction that describes a task in {training_lang} language. Write an response that appropriately completes the request in {training_lang} language. \n'''
    try:
        if lang == 'de':
            text_intro = 'Dies ist der zu bearbeitende Textausschnitt:'
        elif lang == 'en':
            text_intro = 'This is the excerpt to be worked on: '
        training_text += f"### Instruction:\n{prompt_split[0]} \n" + text_intro + f''' {prompt_split[1]} \n### Response: \n'''
    except: 
        training_text += f"### Instruction:\n{prompt_split[0]}\n\n### Response: \n"
        #print(prompt_split)

    training_text += result

    return training_text


def train_df_to_dict(row):
    train_dict = {}
    prompt = row['prompt'] 

    prompt_split = preprocess_prompt(prompt)
    train_dict['instruction'] = prompt_split[0]
    if len(prompt_split) > 1:
        input = prompt_split[1].strip()
        if input[0] == '"':
            input = input[1:]
        if input[-1] == '"':
            input = input[:-1]
        train_dict['input'] = input
    else:
        train_dict['input'] = ''

    
    train_dict['output'] = row['result']

    return train_dict
    




In [ ]:
filepath = './data/annotation/categories_annotation.pkl'

anno_df = 